# Practical work (TP) : Tensile test analysis  

In this session you are asked to analyze data coming from the tensile test.

This analysis aims at extracting mechanical properties of the material, such as the Young modulus, mechanical strength and yield strength from the load/displacement curve.

At the end of this session you should have a class that is able to deal with the data coming from a tensile test. 

## This class must be able to :

### Level 0
- read the data in a csv file
- compute the nominal stress and nominal strain
- compute the true stress and true strain
- plot the relevant curves

### Level 1
- read automatically the header metadata of csv file
- determine the Young modulus $E$
- determine the Yield stress and the Yield stress a 0.02% of strain
- determine the mechanical strength

### Level 2
- compute the plastique strain ($\varepsilon_p = \varepsilon - \dfrac{\sigma}{E} $)
- fit the stress / plastic strain curve with a power plastic law ($\sigma = \sigma_0 + K {\varepsilon_p}^n $) 

### Level 3
- Run the procedure over all the batch.


In [ ]:
# Setup
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

## Steps to structure your work

1- Find the way to extract the relevant data from one dataset (E, Re, Rm, ...)

2- Structure your work using the class structure of python

3- Perform analysis on several loads/displacement curves  




## The dataset :
It can be dowloaded at this link [TensileData](https://drive.google.com/drive/u/1/folders/1Kp-Omga2JykownLw2rureS9A-PhsSJMx).

It contains 4 files, the ***unit are s/N/mm*** :

- Test_1.csv
- Test_2.csv
- Test_3.csv 
- Test_4.csv




## Useful function 

In [ ]:
line = "z = 143.2"
a, b = line.split("=")
print(a, b)

### Here is a simple template of the class that you should build

In [ ]:
class TensileTest:
    """
    this class is able to ....
    """

    def __init__(self, csv_file="xxx"):
        self.csv_file = csv_file
        # CODDE HERE

    def __repr__(self):
        return "<tensileTest: (file: {0}, E = {1:.2} MPa)>".format(
            self.csv_file, self.get_YoungModulus()
        )

    def get_YoungModulus(self):
        # CODDE HERE
        return 1.0

In [ ]:
test1 = TensileTest()  # call the __init__ function

In [ ]:
test1  # call the __repr__ function

In [ ]:
# CODDE HERE